In [2]:
!python3 -m nltk.downloader punkt
!python3 -m nltk.downloader stopwords
!python3 -m nltk.downloader wordnet
!python3 -m nltk.downloader omw-1.4
!python3 -m nltk.downloader averaged_perceptron_tagger
!pip install sentence-transformers

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/lib/p

In [3]:
!git clone https://github.com/facebookresearch/faiss.git

Cloning into 'faiss'...
remote: Enumerating objects: 43446, done.
remote: Counting objects: 100% (3941/3941), done.
remote: Compressing objects: 100% (431/431), done.
remote: Total 43446 (delta 3578), reused 3666 (delta 3503), pack-reused 39505
Receiving objects: 100% (43446/43446), 132.17 MiB | 13.38 MiB/s, done.
Resolving deltas: 100% (39574/39574), done.


In [4]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.5 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.metrics.pairwise import cosine_similarity
import re
import ast


import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
import spacy

data = pd.read_csv('processed_data.csv')
data = data[data['desc'] != 'none']
data = data.reset_index(drop=True)

In [6]:
def tokenize(text):
    # Remove stop words
    tokens = word_tokenize(text)
    return tokens

In [7]:
tokenized_documents = data['desc_tokens'].apply(ast.literal_eval)
documents = [' '.join(doc) for doc in tokenized_documents]

In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

In [9]:
# Training the model on the daaset
encoded_data = model.encode(documents)

In [19]:
import faiss
encoded_data = np.asarray(encoded_data.astype('float32'))
encoded_data = encoded_data / np.linalg.norm(encoded_data, axis=1, keepdims=True)
index = faiss.IndexIDMap(faiss.IndexFlatL2(len(encoded_data[0])))
index.add_with_ids(encoded_data, np.array(range(0, len(encoded_data))))
faiss.write_index(index, 'books_euc.index')



In [30]:
import textwrap
query = 'A fantacy about child who goes to a secret magicians school'
query_vector = model.encode([query])
top_k = index.search(query_vector, 10)

print(f"The Query: {query}","\n")
print("Related Books:")
for i in top_k[1][0]:
  wrapped_paragraph = textwrap.fill(data['desc'][i], width=120)

  print(data['book_title'][i],"\n")
  print(wrapped_paragraph, "\n\n\n")

The Query: ring 

Related Books:
The Lord of the Rings' Metaphors 

 This book is about a rare achievement in history, that of forecasting the present. While JRR Tolkiens epic saga, The
Lord of the Rings, unfolds in an imaginary world set in ancient time, he speaks in metaphor about our time. It is as if
he explored our present world fifty years in advance on the basis of established trends in our responses to universal
principles. In his explorations he sets before us critical choices and asks us to forecast our future according to the
logic of the choices that we are willing to make. -- The exploration in this book of The Lord of the Rings metaphors
assumes that the reader is somewhat familiar with the storyline of Tolkiens great tale, The Lord of the Rings. The book
is designed to enrich this story by bringing into focus surprising elements of it, rather than to retell the story
itself. 



The Lord of the Rings (Literature Guide) 

 This engaging guide is packed with teaching ideas

In [32]:
query = 'a Journey of two young people through an ancient world to destroy a masterious ring'
query_vector = model.encode([query])
top_k = index.search(query_vector, 10)

print(f"The Query: {query}","\n")
print("Related Books:")
for i in top_k[1][0]:
  wrapped_paragraph = textwrap.fill(data['desc'][i], width=120)

  print(data['book_title'][i],"\n")
  print(wrapped_paragraph, "\n\n\n")

The Query: a Journey of two young people through an ancient world to destroy a masterious ring 

Related Books:
The Lord Of The Rings Tarot Book 

 The Lord of the Rings Tarot unites two great traditions: the spiritual, mystical tradition of the tarot, and the world
of folklore and fairy tales that is most delightfully depicted in the works of J.R.R. Tolkien. The added factor of a
card game summons you to travel through Middle-Earth to reach Mount Doom and destroy the One Ring! 



Castaways of the Flying Dutchman 

 A boy and dog trapped aboard the Flying Dutchman, are sent off on an eternal journey by an avenging angel, roaming the
earth throughout the centuries in search of those in need. Their travels lead them to Chapelvale, a sleepy nineteenth
century village whose existence is at stake. Only by discovering the buried secrets and solving the dust-laden riddles
of the ancient village can it be saved.  



Faith Journey Through Fantasy Lands: A Christian Dialogue with Harry Potter,

In [12]:
index